In [1]:
import numpy as np
import pandas as pd

import quandl

# Load Data from Quandl

In [2]:
quandl.ApiConfig.api_key = 'API_KEY'

start_date = '1993-01-01'
end_date = '2021-06-30'

sigs_ticks = ["MULTPL/SP500_DIV_YIELD_MONTH","MULTPL/SP500_EARNINGS_YIELD_MONTH","YC/USA10Y"]
sigs_names = ['DP','EP', 'US10Y']
sigs_info = pd.DataFrame({'Name':sigs_names,'Ticker':sigs_ticks}).set_index('Name')

signals = pd.DataFrame()
for idx,tick in enumerate(sigs_info['Ticker']):
    temp = quandl.get(tick, start_date=start_date, end_date=end_date)
    temp.columns = [sigs_info.index[idx]]
    signals = signals.join(temp,rsuffix='_',how='outer')

# some monthly data reported at start of month--assume we do not have it until end of month
signals = signals.resample('M').last()
signals.columns.name = 'SP500 Multiples'
signals

SP500 Multiples,DP,EP,US10Y
Date,,,
1993-01-31,2.85,4.44,6.39
1993-02-28,2.82,4.44,6.03
1993-03-31,2.77,4.41,6.03
1993-04-30,2.82,4.44,6.05
1993-05-31,2.81,4.38,6.16
...,...,...,...
2021-02-28,1.49,2.42,1.44
2021-03-31,1.47,2.47,1.74
2021-04-30,1.39,2.35,1.65


In [3]:
spy_tick = 'EOD/SPY'
spy = quandl.get(spy_tick, start_date=start_date, end_date=end_date)[['Adj_Close']]
spy_rets = spy.resample('M').last().pct_change()
spy_rets.columns = ['SPY']

rf_tick = 'YC/USA3M'
rf = quandl.get(rf_tick, start_date=start_date, end_date=end_date)
rf = rf.resample('M').last()/(12*100)
rf.rename(columns={'Rate':'US3M'},inplace=True)

In [4]:
rets = spy_rets.join(rf,how='inner')
retsx = rets['SPY'].sub(rets['US3M']).to_frame().rename(columns={0:'SPY'})

# Save Data to Excel

In [ ]:
with pd.ExcelWriter('sp500_fundamentals.xlsx') as writer:  
    signals.to_excel(writer, sheet_name= 'fundamentals')
    rets.to_excel(writer, sheet_name='total returns')
    #retsx.to_excel(writer, sheet_name='excess returns')